#Convolutional Neural Networks
This is short introduction to training convolutional neural networks in PyTorch. We first define the convolutional neural network and training and testing functions.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        w = (32,64) #Number of channels in 1st and 2nd layers
        self.conv1 = nn.Conv2d(1, w[0], 3, 1)
        self.conv2 = nn.Conv2d(w[0], w[1], 3, 1)
        self.fc1 = nn.Linear(w[1]*5*5, 512)
        self.fc2 = nn.Linear(512, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # images start out 28x28
        x = self.conv1(x)  #26x26 images
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  #13x13 images
        x = self.conv2(x)  #11x11 images
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  #5x5 images
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


#stochastic gradient descent
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Now let's run code to download MNIST and run the training.

In [2]:
#Training settings
cuda = True   #Use GPU acceleration (Edit->Notebook Settings and enable GPU)
seed = 1       #Seed for random number generator
batch_size = 64
test_batch_size = 1000
learning_rate = 1.0   #Learning rate
gamma = 0.7     #Learning rate step
epochs = 14
torch.manual_seed(seed)

#GPU
use_cuda = cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

if use_cuda:
    cuda_kwargs = {'num_workers': 1,'pin_memory': True,'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([transforms.ToTensor()])
dataset1 = datasets.MNIST('./data', train=True, download=True,transform=transform)
dataset2 = datasets.MNIST('./data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

#Save model
torch.save(model.state_dict(), "mnist_cnn.pth")

100%|██████████| 9.91M/9.91M [00:11<00:00, 900kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 63.7kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.25MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 8.24MB/s]


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.311258
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.381221
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.124412
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.143522
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.104330
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.095187
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.177269
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.060563
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.205674
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.171758

Test set: Average loss: 0.0453, Accuracy: 9848/10000 (98%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.060170
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.077313
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.016890
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.005444
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.053288
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.100470
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.051439
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.135846
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.034816
T

#Exercises
1. Try another dataset from [Torch Datasets](https://pytorch.org/vision/stable/datasets.html).
2. Try to plot the filters from the second layer, using graphlearning.image_grid.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


class CifarNet(nn.Module):
    def __init__(self):
        super(CifarNet, self).__init__()
        w = (32,64) #Number of channels in 1st and 2nd layers
        self.conv1 = nn.Conv2d(3, w[0], 3, 1)
        self.conv2 = nn.Conv2d(w[0], w[1], 3, 1)
        self.fc1 = nn.Linear(w[1]*6*6, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128,10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        # images start out 28x28
        x = self.conv1(x)  #26x26 images
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  #13x13 images
        x = self.conv2(x)  #11x11 images
        x = F.relu(x)
        x = F.max_pool2d(x, 2)  #5x5 images
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output


#stochastic gradient descent
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

#Training settings
cuda = True   #Use GPU acceleration (Edit->Notebook Settings and enable GPU)
seed = 1       #Seed for random number generator
batch_size = 64
test_batch_size = 1000
learning_rate = 1.0   #Learning rate
gamma = 0.7     #Learning rate step
epochs = 15
torch.manual_seed(seed)

#GPU
use_cuda = cuda and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}

if use_cuda:
    cuda_kwargs = {'num_workers': 1,'pin_memory': True,'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform = transforms.Compose([transforms.ToTensor()])
dataset1 = datasets.CIFAR10('./data', train=True, download=True,transform=transform)
dataset2 = datasets.CIFAR10('./data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = CifarNet().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

#Save model
torch.save(model.state_dict(), "cifar10_cnn.pth")


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.311228
Train Epoch: 1 [6400/50000 (13%)]	Loss: 2.184641
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.994224
Train Epoch: 1 [19200/50000 (38%)]	Loss: 1.838387
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.778403
Train Epoch: 1 [32000/50000 (64%)]	Loss: 1.833293
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.559589
Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.656407

Test set: Average loss: 1.6761, Accuracy: 4078/10000 (41%)

Train Epoch: 2 [0/50000 (0%)]	Loss: 1.871727
Train Epoch: 2 [6400/50000 (13%)]	Loss: 1.521663
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.372251
Train Epoch: 2 [19200/50000 (38%)]	Loss: 1.029970
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.234065
Train Epoch: 2 [32000/50000 (64%)]	Loss: 1.201975
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.352843
Train Epoch: 2 [44800/50000 (90%)]	Loss: 0.976336

Test set: Average loss: 1.1442, Accuracy: 5957/10000 (60%)

Train Epoch: 3 [0/50000 (0%)]	Loss: 1.148339
Train Epoch: 3 [6400/50000 (13%)]	Loss: 1.158

Below is the solution to the second exercise.

In [1]:
!pip install -q graphlearning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import numpy as np
import graphlearning as gl

w2 = model.conv2.weight.cpu().detach().numpy()
print(w2.shape)

I = np.reshape(w2[:,16,:,:], (64,9))
print(I.shape)
gl.utils.image_grid(I,n_rows=8,n_cols=8)
I = np.reshape(np.sum(np.absolute(w2),axis=1), (64,9))
print(I.shape)
gl.utils.image_grid(I,n_rows=8,n_cols=8)

NameError: name 'model' is not defined